In [ ]:
#!pip install -U --no-cache-dir gdown --pre
import gdown

In [ ]:
%cd '/content/'
!mkdir 'blender_images'
%cd '/content/blender_images'

# 24 images
# https://drive.google.com/file/d/1gL0JuKRz8sORNROpWGnmugGGybCIC8G7/view?usp=drive_link
#url = 'https://drive.google.com/uc?id=1gL0JuKRz8sORNROpWGnmugGGybCIC8G7'

# https://drive.google.com/file/d/1O1oB-wpN8uUYI3tRxyukqqDstaQA2OeI/view?usp=drive_link
# Toy plane
# https://drive.google.com/file/d/1yimVSn49rvJT-vcSvfJjhE9R4n0YY-Gp/view?usp=drive_link

url = 'https://drive.google.com/uc?id=1yimVSn49rvJT-vcSvfJjhE9R4n0YY-Gp'

output = 'toy_plane.tar.gz'
gdown.download(url, output, quiet=False)

!tar -xzf toy_plane.tar.gz

In [ ]:
# Install library
%cd '/content/'
!wget 'https://github.com/AcademySoftwareFoundation/Imath/releases/download/v3.1.11/Imath-3.1.11.tar.gz'
!tar -xf 'Imath-3.1.11.tar.gz'
!mkdir 'imath_build'
!mkdir '/content/imath_install'
%cd 'imath_build'
!cmake '/content/Imath-3.1.11' --install-prefix '/content/imath_install'
!make install

!pip install open3d
!pip install idebug
!pip install openexr

In [ ]:
import Imath
import OpenEXR
import argparse
import array
import numpy as np
import os
from open3d import *
from open3d.geometry import Image
from tqdm.notebook import tqdm

In [ ]:
def read_exr(exr_path, height, width):
    file = OpenEXR.InputFile(exr_path)
    depth_arr = array.array('f', file.channel('R', Imath.PixelType(Imath.PixelType.FLOAT)))
    depth = np.array(depth_arr).reshape((height, width))
    depth[depth < 0] = 0
    depth[np.isinf(depth)] = 0
    depth[depth > 10] = 0
    return depth

def depth2pcd(depth, intrinsics, pose):
    inv_K = np.linalg.inv(intrinsics)
    inv_K[2, 2] = -1
    depth = np.flipud(depth)
    y, x = np.where(depth > 0)
    # image coordinates -> camera coordinates
    points = np.dot(inv_K, np.stack([x, y, np.ones_like(x)] * depth[y, x], 0))
    # camera coordinates -> world coordinates
    points = np.dot(pose, np.concatenate([points, np.ones((1, points.shape[1]))], 0)).T[:, :3]
    return points

def selectPoints(all_points):
  normed_points_all = all_points-all_points.mean(axis=0)
  unit_normed_points_all = (normed_points_all - normed_points_all.min(axis=0)) / (normed_points_all.max(axis=0) - normed_points_all.min(axis=0))
  normed_points_all = unit_normed_points_all - 0.5
  indices_selected = np.random.choice(np.arange(len(normed_points_all)), 16384, replace=False)
  points_selected = normed_points_all[indices_selected,:]
  return points_selected

In [ ]:
N_POINT = 2048

def depth2pcdNotNormed(depth, intrinsics, pose):
    inv_K = np.linalg.inv(intrinsics)
    inv_K[2, 2] = -1
    depth = np.flipud(depth)
    y, x = np.where(depth > 0)
    # image coordinates -> camera coordinates
    points = np.dot(inv_K, np.stack([x, y, np.ones_like(x)] * depth[y, x], 0))
    return points.T[:, :3]

def selectPointsWithoutNorm(all_points):
  indices_selected = np.random.choice(np.arange(len(all_points)), N_POINT, replace=False)
  points_selected = all_points[indices_selected,:]
  return points_selected

In [ ]:
output_directory = os.path.join('/content','rendered_pcd_denormalized')
os.makedirs(output_directory, exist_ok=True)

In [ ]:
import json
import open3d as o3d

all_images_dir = os.path.join('/content','blender_images')

for sample_name in tqdm(os.listdir(all_images_dir)):
    if '.tar' in sample_name:
        continue
    sample_directory = os.path.join('/content','blender_images', sample_name, 'render')
    if not os.path.isdir(sample_directory):
        continue
    transform_json_path = os.path.join(sample_directory, 'transforms.json')
    depth_images_path = os.path.join(sample_directory, 'depths')


    with open(transform_json_path, 'r') as jfile:
      json_content = json.load(jfile)

    depth_images_names = os.listdir(depth_images_path)
    for depth_image_name in depth_images_names:
      exr_path = os.path.join(depth_images_path, depth_image_name)
      frame_name = depth_image_name.replace('_depth.exr', '')
      frame_detail = [content for content in json_content['frames'] if content['file_path'] == frame_name][0]

      depth = read_exr(exr_path, 800, 800)
      depth = depth*1000
      depth_img = Image(np.uint16(depth))

      H = W = 800
      camera_angle_x = json_content['camera_angle_x']
      focal = .5 * W / np.tan(.5 * camera_angle_x)
      camera_intrinsics = np.array([[focal, 0, H], [0, focal, W], [0,0,1]])
      pose = frame_detail['transform_matrix']

      points_all = depth2pcdNotNormed(depth_img, camera_intrinsics, pose)
      points_selected = selectPointsWithoutNorm(points_all)

      pcd = o3d.geometry.PointCloud()
      pcd.points = o3d.utility.Vector3dVector(points_selected)

      sample_output_directory = os.path.join(output_directory, sample_name)
      os.makedirs(sample_output_directory, exist_ok=True)

      open3d.io.write_point_cloud(os.path.join(sample_output_directory, '%s.pcd' % frame_name ), pcd)

In [ ]:
pcd = o3d.io.read_point_cloud('/content/rendered_pcd_denormalized/toy_plane_005/r_1.pcd')

In [ ]:
o3d.visualization.draw_plotly([pcd])

In [ ]:
!pip install openxlab

In [ ]:
!openxlab login

In [ ]:
!openxlab dataset ls --dataset-repo omniobject3d/OmniObject3D-New

In [ ]:
!openxlab dataset download --dataset-repo omniobject3d/OmniObject3D-New --source-path /raw/blender_renders/toy_plane.tar.gz

In [ ]:
%cd /content
!openxlab dataset download --dataset-repo omniobject3d/OmniObject3D-New --source-path /raw/blender_renders/toy_car.tar.gz

In [ ]:
%cd /content/omniobject3d___OmniObject3D-New/raw/blender_renders

In [ ]:
!tar -xzf toy_plane.tar.gz

In [ ]:
!tar -xzf toy_car.tar.gz